<a href="https://colab.research.google.com/github/LaysaBit/Agrupamento-Espectral/blob/main/Clusterizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Modules needed

#### 1.1 To acess Littleballoffur module

In [7]:
import os, sys 
#to be able to interact with Google Drive's operating system
from google.colab import drive 
#drive is a module that allows us use Python to interact with google drive
drive.mount('/content/gdrive') 
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#!pip uninstall scikit-learn -y

!pip install -U scikit-learn

#### 1.2 The other modules

In [8]:
import copy
import os
import math as mt
import random
import pandas as pd
import networkx as nx        
import numpy as np
import matplotlib.pyplot as plt
from littleballoffur import RandomWalkSampler
from littleballoffur import RandomNodeSampler
from littleballoffur import RandomEdgeSampler
import sklearn
from sklearn import metrics
from sklearn.metrics import rand_score

EPSILON = 0.0000001 

### Arquivo do Grafo

In [37]:
!git clone https://github.com/lauraassis/CDP

fatal: destination path 'CDP' already exists and is not an empty directory.


### Criando o grafo

In [43]:
def build_graph(graph, path):

    prelim = []                          # Lista auxiliar da função. 
    coor = []                            # Uma lista de coordenadas, caso seja preciso posteriormente.
    clusters_from_fil = []               # Vai guardar os clusters descritos no arquivo.
    i = 0

    file1 = open(""+path+"", mode="r", encoding='UTF-8')
    line = file1.readline().split()
    line = [int(line[x]) for x in range(len(line)-1)]
    vertices = line[0]

    for line in file1:
        prelim = line.split()

        if( i < vertices ):
          if(prelim != []):
              coor.append((int(prelim[0]),int(prelim[1])))         # Preenchendo a lista de coordenadas. 
              graph.add_node(i, X = coor[i][0], Y = coor[i][1])    # Adiciona atributos aos vértices.
              clusters_from_fil.append( int(prelim[2]) )           # Guarda os clusters do arquivo.
              i+=1
        else:
          if(prelim != []):
              graph.add_edge(int(prelim[0]), int(prelim[1]))       # Adiciona as arestas do grafo

    file1.close()

    return coor, clusters_from_fil

### Separar os vértices de acordo com as entradas do vetor de Fiedler

Parâmetros:
*   Vetor de Fiedler
*   Grafo



In [10]:
def get_vertices(vector, graph):
    neg = []
    pos = []
    tot = []
    GraphNodes = list(graph.nodes())
    
    for i in range(len(vector)):
        if (vector[i] < 0):
            neg.append(GraphNodes[i])
        else:
            pos.append(GraphNodes[i])
            
    tot.append(neg)
    tot.append(pos)
    return tot

#### Busca o número de vértices e de arestas de cada cluster

In [11]:
def get_NumVNumE(lista_in, graph):
    lista_out = []
    for i in range(len(lista_in)):
        GraphInf = []
        
        H = nx.Graph()
        H = copy.deepcopy(graph.subgraph(lista_in[i]))
        
        nvert = H.number_of_nodes()
        GraphInf.append(nvert)
        
        nedges = H.number_of_edges()
        GraphInf.append(nedges)
        
        lista_out.append(GraphInf)
    return lista_out

#### Densidade do cluster; 

In [12]:
def get_density(lista_in):
  #lista_out = []          # Mudar isso aqui.
  lista_out = [v[1]/v[0] for v in lista_in]
  return lista_out

#### Calcula o vetor de Fiedler

In [13]:
def compute_fiedler(matrix):
    val,w = np.linalg.eigh(matrix)
    vet_out = [w[i][1] for i in range(len(w))]

    return vet_out

#### Clusteriza o grafo e armazena os vértices do cluster na lista 

In [14]:
def clusterizacao(graph, size, clusters, ListOfGraphs):
    Lap = nx.laplacian_matrix(graph).toarray()  
    L = [ list(Lap[i]) for i in range(len(graph)) ]
    
    Lap

    fv = compute_fiedler(L)

    #Evita erros numéricos
    for i in range(len(fv)):
        if (abs(fv[i]) <= EPSILON): 
            fv[i] = 0

    vec = get_vertices(fv, graph)

    #Separa os subgrafos de acordo com o vetor de Fiedler
    H1 = copy.deepcopy(graph.subgraph(vec[0]))
    H2 = copy.deepcopy(graph.subgraph(vec[1]))

    #Organiza os subgrafos de acordo com seus tamanhos
    if (H1.number_of_nodes() > size):
        if(nx.number_connected_components(H1) == 1):
            ListOfGraphs.append(H1)
        else:
            for c in (nx.connected_components(H1)):
                H = copy.deepcopy(H1.subgraph(c))
                if (H.number_of_nodes() > size):
                    ListOfGraphs.append(H)
                else:
                    clusters.append(list(H.nodes()))
                    
    else:
        if(len(vec[0]) != 0):
            clusters.append(vec[0])
        

    if (H2.number_of_nodes() > size):
        if(nx.number_connected_components(H2) == 1):
            ListOfGraphs.append(H2)
        else:
            for c in nx.connected_components(H2):
                H = copy.deepcopy(H2.subgraph(c))
                if (H.number_of_nodes() > size):
                    ListOfGraphs.append(H)
                else:
                    clusters.append(list(H.nodes()))
                    
    else:
        if (len(vec[1]) != 0):
            clusters.append(vec[1])
    
    return clusters, ListOfGraphs

#### Construção dos clusters

In [15]:
def BuildCluster(graph, size):
    ListOfGraphs = [graph]
    clusters     = []
    
    while (len(ListOfGraphs) != 0):
        tested_graph = ListOfGraphs[0]
        ListOfGraphs.pop(0)
        clusters, ListOfGraphs = clusterizacao(tested_graph, size, clusters, ListOfGraphs)

    return clusters

#### Desenha um cluster específico

Parâmetros:
*   lista de vertices do cluster;
*   grafo original;
*   cluster específico.





In [16]:
def draw_cluster(lista_in, graph, c, mapa):
    plt.figure(figsize=(8, 8))
    
    if ( (c < 0) or (c > len(lista_in))):
        print('cluster inexistente')
        return
    else:
        H = copy.deepcopy(graph.subgraph(lista_in[c]))

    map_aux = [ mapa[lista_in[c][i]-1] for i in range(len(lista_in[c]))]
    nx.draw(H, with_labels=True, node_color = map_aux, node_size = 900, font_size = 10, linewidths = 0.5, edge_color = 'black')
    plt.show()

#### Atribuir cores aos vértices.



*   atribui a cada cluster uma cor selecionada aleatoriamente.





In [17]:
def set_color_node(graph, lista_in):
  mapa_cores = np.zeros((len(graph),), dtype = tuple)

  for i in range(len(lista_in)): 
      color = (random.randint(0,255)/255, random.randint(0,255)/255, random.randint(0,255)/255, 1.0)
      for node in lista_in[i]:  
          mapa_cores[node-1] = color
  
  return mapa_cores

#### Grafo de Corte 

In [18]:
def GetCutGraph(graph, lista_in):
    cGraph = nx.Graph()
    cGraph = copy.deepcopy(graph)
    for c in lista_in:
        H = graph.subgraph(c)
        for e in H.edges():
            e1 = (e[1],e[0])
            for f in graph.edges():
                if (e == f):
                    cGraph.remove_edge(e[0], e[1])
                elif (e1 == f):
                    cGraph.remove_edge(e1[0], e1[1])
    return cGraph

####  Qualidade da clusterização

In [19]:
def measuring_partition(graph, lista_in): #n utiliza as coordenadas
    measure = []
    cov = nx.algorithms.community.coverage(graph,lista_in)
    measure.append(cov)
    per = nx.algorithms.community.performance(graph,lista_in)
    measure.append(per)
    mod = nx.algorithms.community.modularity(graph,lista_in)
    measure.append(mod)
    return measure

#### Criando Grafo, Clusters

*   nv = número de vértices
*   ne = número de arestas
*   cluster_size = se é direcionado (Verificar com João se é isso mesmo)





In [31]:
def get_parameters(path):
  
  file1 = open("'"+path+"'", mode="r", encoding='UTF-8')
  line = file1.readline().split()
  line = [int(line[x]) for x in range(len(line)-1)]

  vert, edges,cluster_size = line[0:3]
  print(vert, edges,cluster_size)                       #Test

  return vert, edges, cluster_size

In [44]:
      G         = nx.Graph()
      cut_graph = nx.Graph()    
      coor = []
      clusters_from_file = []
      
      nv, ne, cluster_size = get_parameters("/content/CDP/T1/A1100a.txt")
      
      if (cluster_size > 0):
          coor, clusters_from_file = build_graph(G, "/content/CDP/T1/A1100a.txt")
      else:
          print("Cluster size error!")
      
      originalCluster   = BuildCluster(G, cluster_size)
        
      Cluster_NumV_NumE = get_NumVNumE(originalCluster,G)
      ClusterDensity = get_density(Cluster_NumV_NumE)

      #color_map = set_color_node(G,originalCluster)
      #cut_graph = GetCutGraph(G,originalCluster)
      
      # m = measuring_partition(G,originalCluster)
      # print('Cobertura: '   , round(m[0],2))           #Test
      # print('Performance: ' , round(m[1],2))           #Test
      # print('Modularidade: ', round(m[2],2))           #Test

      # print('cluster_size: ', cluster_size)            #Test
      # print(cut_graph.edges())                         #Test
      # print(Cluster_NumV_NumE)                         #Test
      # print(ClusterDensity)                            #Test
      # print(originalCluster)                           #Test

512 976 10


### Plottings do Supergrafo

#### 1.1 Desenha Grafo

In [ ]:
# plt.figure(figsize=(8, 8))
# pos = nx.spring_layout(G, seed = 1)       # Adicionar as coordenadas no arquivo original do grafo.
# nx.draw(G, with_labels=True, pos = pos, node_color = "orange", node_size = 900, linewidths = 0.2, 
#         edge_color = 'black',width = 1.0, font_color = 'blue')
# plt.show()

#### 1.2 Desenha Grafo com Clusters

In [ ]:
# plt.figure(figsize=(8, 8))

# options = {'with_labels': True, 
#             'node_color' : color_map, 
#             'node_size' : 1100,
#             'font_size' : 10, 
#             'linewidths' : 1.0,
#             'edge_color' : 'black'}
# shells = originalCluster
# #nx.draw_shell(G, nlist=shells, **options)

# nx.draw(G, with_labels=True, pos = pos, node_color = color_map, node_size = 900, font_size = 10, width = 1.5, edge_color = 'gray')
# plt.show()

#### 1.3 Desenha os clusters



In [ ]:
# for i in range(0, len(originalCluster)):
#           draw_cluster(originalCluster, G, i, color_map) 

#### 1.4 Desenha Grafo de Corte

In [ ]:
# plt.figure(figsize=(8, 8))
# options = {'with_labels': True, 
#             'node_color' : color_map, 
#             'node_size' : 1000,
#             'font_size' : 10, 
#             'linewidths' : 1.5,
#             'edge_color' : 'gray'}
# shells = originalCluster
# #nx.draw_shell(cut_graph, nlist=shells, **options)
# nx.draw(cut_graph, pos = pos, with_labels=True, node_color = color_map, node_size = 900, width = 2.0, edge_color = 'gray')
# plt.show()

### Criando o Subgrafo Induzido

In [ ]:
    G.add_edge(0,1) # Desconsiderar matematicamente esse edge, adicionado pois numeric_indices = node_indices em RWS sample()
    
    #Selecionar vertice antes de clusterizar
    
    
    # Criar o subgrafo
    model = RandomWalkSampler(number_of_nodes=1000)
    subgraph1 = model.sample(G) 

    # Criar clusters do subgrafo
    subgraphCluster = BuildCluster(subgraph1, cluster_size)
       

    subCluster_NumV_NumE = get_NumVNumE(subgraphCluster, G)
    subClusterDensity    = get_density(subCluster_NumV_NumE)

    print(" Supergraphgraph Numv_NumE ", Cluster_NumV_NumE)
    print(" Supergraph density ", ClusterDensity)
    print(" Subgraph Numv_NumE ", subCluster_NumV_NumE)
    print(" Subgraph density ", subClusterDensity)

    print()
    n = measuring_partition(subgraph1, subgraphCluster)
    print("Subgrafo Induzido:")
    print('Cobertura: '   , round(n[0],2))
    print('Performance: ' , round(n[1],2))
    print('Modularidade: ', round(n[2],2))

    # subColor_map = set_color_node(subgraph, subgraphClusters)
    # subCut_graph = GetCutGraph(subgraph, subgraphClusters)

    # print("Quantidade de clusters no subgrafo", len(subgraphCluster))       #Test
    # print("Quantidade de vértices do subgrafo:", len(subgraph.nodes()))     #Test
    # print("Cardinalidade set do supergrafo:" , len(supergraph_set ))        #Test
    # print("Cardinalidade set do subgrafo:", len(subgraph_set) )             #Test
    
    # print( supergraph_set )                                                 #Test
    # print( subgraph_set)                                                    #Test
    
    # print("O tamanho do clusters do subgrafo é ", len(subgraphClusters))    #Test
    # print("Quantidade de clusters no supergrafo", len(originalCluster))     #Test

 Supergraphgraph Numv_NumE  [[111, 125], [98, 113], [334, 497], [226, 311], [244, 355], [146, 180], [204, 262], [271, 394], [366, 556]]
 Supergraph density  [1.1261261261261262, 1.153061224489796, 1.4880239520958083, 1.3761061946902655, 1.4549180327868851, 1.2328767123287672, 1.2843137254901962, 1.4538745387453875, 1.5191256830601092]
 Subgraph Numv_NumE  [[219, 308], [241, 347], [540, 929]]
 Subgraph density  [1.4063926940639269, 1.4398340248962656, 1.7203703703703703]

Subgrafo Induzido:
Cobertura:  0.73
Performance:  0.6
Modularidade:  0.33


### Plottings do subgrafo induzido

#### 2.1 Desenhando o subgrafo induzido

In [ ]:
# plt.figure(figsize=(8, 8))
# pos = nx.spring_layout(subgraph)
# nx.draw(subgraph, with_labels=True, pos = pos, node_color = "orange", node_size = 900, linewidths = 0.2, 
#         edge_color = 'black',width = 1.0, font_color = 'blue')
# plt.show()


#### 2.2 Desenha subgrafo com clusters

In [ ]:
# plt.figure(figsize=(8, 8))
# options = {'with_labels': True, 
#             'node_color' : subColor_map, 
#             'node_size' : 1100,
#             'font_size' : 10, 
#             'linewidths' : 1.0,
#             'edge_color' : 'black'}
# #shells = originalCluster
# #nx.draw_shell(G, nlist=shells, **options)              #Usa-se ou a linha 272 ou a 273.
# nx.draw(subgraph, with_labels=True, pos = pos, node_color = subColor_map, node_size = 900, font_size = 10, width = 1.5, edge_color = 'gray')
# plt.show()

#### 2.3 Plot clusters do subgrafo induzido 

In [ ]:
#  for i in range(0, len(subgraphClusters)):
#         draw_cluster(subgraphClusters, subgraph, i, subColor_map) 


#### 2.4 Grafo de corte do induzido

In [ ]:
# plt.figure(figsize=(8, 8))
# options = {'with_labels': True, 
#             'node_color' : subColor_map, 
#             'node_size' : 1000,
#             'font_size' : 10, 
#             'linewidths' : 1.5,
#             'edge_color' : 'gray'}
# #shells = originalCluster
# #nx.draw_shell(cut_graph, nlist=shells, **options)
# nx.draw(subCut_graph, pos = pos, with_labels=True, node_color = subColor_map, node_size = 900, width = 2.0, edge_color = 'gray')
# plt.show()

  ### Rand Index
    



In [ ]:
a = len(subgraph1.nodes())
indexSubgraphCluster = np.zeros(shape=a, dtype=int)
indexOriginalCluster = np.zeros(shape=a, dtype=int)

c = 0
a = len(subgraphCluster)
for i in range(0,a):
    b = len(subgraphCluster[i]) 
    for j in range(0,b):
        indexSubgraphCluster[c] = i
        c+=1      

print(indexSubgraphCluster)

nodesubgraph = list(subgraph1.nodes())
a = len(nodesubgraph)
b = len(originalCluster)
c = 0
flag = 0 
for i in range(0,a):
    d = nodesubgraph[i]
    
    for j in range(0,b):
        e = len(originalCluster[j])
        for k in range(0,e):
            if(originalCluster[j][k] == d):
                indexOriginalCluster[c] = j
                c+=1
                flag = 1
                break
        
        if(flag == 1):
            flag = 0           
            break

print(indexOriginalCluster)

c = rand_score(indexOriginalCluster,indexSubgraphCluster)
print()
print(c)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

### Cardinalidade dos clusters (Rascunho)

In [ ]:
# # Random Walk Sampling
#     for i in range(100, 1000):
#       model = RandomWalkSampler(number_of_nodes=i)
#       subgraph = model.sample(G)
#       subgraphCluster = BuildCluster(subgraph, cluster_size)
#     #   subgraph_set = [ tuple(lst) for lst in subgraphCluster]   # Tentar produzir o mesmo resultado usando apenas lista.
#     #   subgraph_set = [ set(tupl) for tupl in subgraph_set]
#       if(len(subgraphCluster)==3):
#           cardinalidade_3.append(i)
#       elif(len(subgraphCluster)==2):
#           cardinalidade_2.append(i)
#     print("Cardinalidade = 3,", len(cardinalidade_3))
#     print(cardinalidade_3)
#     print()                                                     # Melhora na modelagem do resultado se posto em dict
#     print("Cardinalidade = 2,", len(cardinalidade_2))
#     print(cardinalidade_2)
#     print()
#     print("Números de vértices com outras cardinalidades:", 900-len(cardinalidade_3)-len(cardinalidade_2) )

# # Modificação pra conjunto
# supergraph_set = [ tuple(lst) for lst in originalCluster]
# supergraph_set = [ set(tupl) for tupl in supergraph_set]

# subgraph_set = [ tuple(lst) for lst in subgraphCluster]
# subgraph_set = [ set(tupl) for tupl in subgraph_set]

# A = set([1,2,3,4,5,10,11,15])
# B = set([1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,15])
# print(len(A-B))

# result1 = {}
# for i in range(len(supergraph_set)):
#   result[i] = [len(supergraph_set[i]-set2) for set2 in subgraph_set ] 
# print(result1) 


### Rascunho

T4
T11
do T1 ao T12

Analise de topologia
e com base em A e B (tam de nó)

Pegar primeirio grafo do T1